# 5 - Text generation

One of the most advertised features of transformer-based language models is their ability to generate text that is almost indistinguishable from human-written text. A famous example is OpenAi's GPT-2, which when given the prompt:

```
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
```

was able to generate a compelling news article about talking unicorns:

```
The scientist named the population, after their distinctive horn, Ovid’s Unicorn. These four-horned, silver-white unicorns were previously unknown to science. Now, after almost two centuries, the mystery of what sparked this odd phenomenon is finally solved. Dr. Jorge Pérez, an evolutionary biologist from the University of La Paz, and several companions, were exploring the Andes Mountains when they found a small valley, with no other animals or humans. Pérez noticed that the valley had what appeared to be a natural fountain, surrounded by two peaks of rock and silver snow. Pérez and the others then ventured further into the valley. “By the time we reached the top of one peak, the water looked blue, with some crystals on top,” said Pérez. Pérez and his friends were astonished to see the unicorn herd. These creatures could be seen from the air without having to move too much to see them—they were so close they could touch their horns. While examining these bizarre creatures the scientists discovered that the creatures also spoke some fairly regular English …
```

What makes this example so remarkable is that it was generated without any explicit supervision! By simply learning to predict the next word in the text of millions of web pages, GPT-2 and its more powerful descendants like GPT-3 are able to acquire a broad set of skills and pattern recognition abilities that can be activated with different kinds of input prompts. 

The following image shows how language models are sometimes exposed during pretraining to sequences of tasks where they need to predict the following tokens based on the context alone. Some of this tasks include: arithmetics, translation, fixing misspellings, etc

<img src="images/tasks_examples.png" title="" alt="" width="700" data-align="center">

The ability of transformers to generate realistic text has led to a diverse range of applications, auto-completition features like [Write With Transformer](https://transformer.huggingface.co/), text-based games such as [AI dungeon](https://play.aidungeon.io/), and conversational agents like [Google's Meena](https://ai.googleblog.com/2020/01/towards-conversational-agent-that-can.html)

In this chapter, we'll use GPT-2 to illustrate how text generation works for language models and explore how different decoding strategies impact the generated texts.

## 5.1 - The challenge with generating coherent text

Up to this point, we have focused on tackling NLP tasks via a combination of pretraining and supervised fine-tuning. As we have seen, for task-specific heads like sequence or token classification, generating predictions is fairly straightforward; the model produces some logits and we take the maximum value to get the predicted class. By contrast, converting the model's probabilistic output to text requires a **decoding method**, which introduces a few challenges that are unique to text generation:

* The decoding is done **iteratively** and thus involves significantly more compute than simply passing inputs once through the forwards pass of a model.
* The **quality** and **diversity** of the generated text depend on the choice of decoding method and associated hyperparameters.

To understand how this decoding process works, let's start by examining how GPT-2 is pretrained and subsequently applied to generate text.

Like other autoregressive or causal language models, GPT-2 is pretrained to estimate the probability $P(y|x)$ of a sequence of tokens $\mathbf{y} = y_{1}, \dots, y_{t}$, ocurring in the text, given some initial prompt or context sequence $\mathbf{x} = x_{1}, \dots, x_{k}$. Since it is impractical to adcquire enough training data to estimate $P(\mathbf{y}|\mathbf{x})$ directly, it is common to use the chain rule of probability to factorize it as a product of conditional probabilities:

$$
P(y_{1}, \dots, y_{t}| \mathbf{x}) = \prod_{t=1}^{N} P(y_{t}|y_{1}, \dots, y_{t-1}, \mathbf{x})
$$

It is from these conditional probabilities that we pick up the intuition that autoregressive language modeling amounts to predicting each word given the preceding words in a sentence; this is exactly what the probability on the righthand side of the preceding equation describes. <span style="color:blue">Notice that this pretraining objective is quite different from BERT's, which utilizes both <b>past</b> and <b>future</b> contexts to predict a *masked* token.</span>

<img src="images/text_generation_example.png" title="" alt="" width="500" data-align="center">

As shown by the figure, we start with a prompt like "Transformers are the" and use the model to predict the next token. Once we have determined the next token, we append it to the prompt and then use the new input sequence to generate another token. We do this until we have reached a special end-of-sequence token or a predefined maximum length.

----

**Note:** Since the output sequence is *conditioned* on the choice of input prompt, this type of text generation is often called <span style="color:blue">conditional text generation</span>.

----

At the heart of this process lies a decoding method that determines which token is selected at each timestep. Since the language model head produces a logit $z_{t,i}$ per token in the vocabulary at each step, we can get the probability distribution over the next possible token $w_{i}$ by taking the softmax:

$$
P(y_{t} = w_{i} | y_{1}, \dots, y_{t-1}, \mathbf{x}) = \text{softmax}(z_{t,i})
$$

The goal of most decoding methods is to search for the most likely overall sequence by picking a $\hat{\mathbf{y}}$ such that:

$$
\hat{\mathbf{y}} = \underset{\mathbf{y}}{argmax} \ P(\mathbf{y} | \mathbf{x})
$$

<span style="color:blue">Finding</span> $\hat{\mathbf{y}}$ <span style="color:blue">directly would involve <b>evaluating every possible sequence</b> with the language model. Since there does not exist an algorithm that can do this in a reasonable amount of time, <b>we rely on approximations instead</b></span>.

## 5.2 - Decoding methods

[**Great article from hugginface on decoding methods**](https://huggingface.co/blog/how-to-generate)

### 5.2.1 - Greedy search decoding

<img src="images/greedy_search_tree.png" title="" alt="" width="400" data-align="center">

This is the simplest decoding method to get discrete tokens from a model's continuous output is to greedily select the token with the highest probability at each timestep:

$$
\hat{y_{t}} = \underset{y_{t}}{argmax} \ P(y_{t} | y_{1}, \dots, y_{t-1}, \mathbf{x})
$$

To see how greedy search works, let's start by loading the GPT-2 model with a language modeling head:

In [1]:
# https://huggingface.co/transformers/v2.2.0/pretrained_models.html

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2" # 117M parameters
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

ModuleNotFoundError: No module named 'torch'

Now, let's generate some text! Although 🤗 Transformers provides a `generate()` function for autoregressive models like GPT-2, we'll implement this decoding method ourselves to see what goes under the hood. To warm up, we'll use "Transformers are the" as the input prompt and run the decoding for eight timesteps. At each timestep, we pick out the model's logits for the last token in the prompt and wrap them with a softmax to get a probability distribution. We then pick the next token with the highest probability, add it to the input sequence, and run the process again.

In [ ]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
            
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

pd.DataFrame(iterations)

With this simple method we were able to generate the sentence `Transformers are the most popular toy line in the world`. Interestingly, this indicates that GPT-2 has internalized some knowledge about the Transformers media franchise, which was created by two toy companies (Hasbro and Takara Tomy).

Unlike other tasks such as sequence classification where a single forward pass suffices to generate predictions, with text generation we need to decode the output tokens one at a time.

While implementing greedy search was not too hard, it would be better to use the built-in `generate()` function from 🤗 Transformers to explore more sophisticated decoding methods. To reproduce our example, let's make sure sampling is switched off (it’s off by default, unless the specific configuration of the model you are loading the checkpoint from states otherwise) and specify the `max_new_tokens` for the number of newly generated tokens:

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

Now, let's try something a bit more interesting: can we reproduce the unicorn story from OpenAI? As we did previously, we'll encode the prompt with the tokenizer, and we'll specify a larger value for `max_length` to generate a longer sequence of text:

In [ ]:
max_length = 128

input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)

print(tokenizer.decode(output_greedy[0]))

From the resulting text we can see that one of the main drawbacks with greedy search decoding is that it tends to produce repetitive output sequences, which is certainly undesirable in a news article. This is a common problem with greedy search algorithms, which can fail to give you the optimal solution; in the context of decoding, <span style="color:blue">they can miss word sequences whose overall probability is higher <b>just because words happen to be preceded by low probability ones.</b></span>

----

**Note:** Although greedy search decoding is rarely used for text generation that require diversity (see this interesting video on that matter), <span style="color:blue">it can be useful for producing short sequences like arithmetic where a determinisit and factually correct output is preferred.</span>

----

### 5.2.2 - Beam search decoding